In [1]:
#Imports and Dependencies

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Build the Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation="relu", input_shape=(100,)),
        layers.Dense(784, activation="tanh"),
        layers.Reshape((28, 28))
    ])
    return model
#Build the Generator
# Build the Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    return model


In [5]:
# Load and Normalize Dataset (MNIST)
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 127.5 - 1
x_train = np.expand_dims(x_train, axis=-1)

# (x_train, _), _ = tf.keras.datasets.mnist.load_data()
# x_train = x_train.astype("float32") / 127.5 - 1
# x_train = np.expand_dims(x_train, axis=-1)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Dataset Pipeline

BUFFER_SIZE = 60000
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [7]:
# Initialize Models and Optimizers

generator = build_generator()
discriminator = build_discriminator()

cross_entropy = tf.keras.losses.BinaryCrossentropy()
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
disc_optimizer = tf.keras.optimizers.Adam(1e-4)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
#Training Step (Single Batch)

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100])
    #Forward & Backward Pass

    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        #Compute Losses
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + cross_entropy(tf.zeros_like(fake_output), fake_output)
        # Backpropagation
        gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        gen_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
        disc_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))

        return gen_loss, disc_loss

In [9]:
def train(dataset, epochs=100):
    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)
        print(f"Epoch {epoch+1}/{epochs} | Gen Loss: {gen_loss.numpy():.4f} | Disc Loss: {disc_loss.numpy():.4f}")
    generator.save("mnist_gan_generator.keras")  # Use .keras instead of .h5

# 4. Actually train the GAN!
train(train_dataset, epochs=50)

# 5. Load the trained model (for Gradio)
generator = tf.keras.models.load_model("mnist_gan_generator.keras")


Epoch 1/50 | Gen Loss: 0.7460 | Disc Loss: 0.6806
Epoch 2/50 | Gen Loss: 1.1456 | Disc Loss: 0.4626
Epoch 3/50 | Gen Loss: 1.4344 | Disc Loss: 0.3690
Epoch 4/50 | Gen Loss: 1.2707 | Disc Loss: 0.4835
Epoch 5/50 | Gen Loss: 1.0902 | Disc Loss: 0.6629
Epoch 6/50 | Gen Loss: 1.0503 | Disc Loss: 0.6598
Epoch 7/50 | Gen Loss: 1.0425 | Disc Loss: 0.6753
Epoch 8/50 | Gen Loss: 1.0255 | Disc Loss: 0.7797
Epoch 9/50 | Gen Loss: 1.0387 | Disc Loss: 0.7210
Epoch 10/50 | Gen Loss: 0.8709 | Disc Loss: 0.9729
Epoch 11/50 | Gen Loss: 1.1478 | Disc Loss: 0.7000
Epoch 12/50 | Gen Loss: 1.0857 | Disc Loss: 0.7291
Epoch 13/50 | Gen Loss: 1.3577 | Disc Loss: 0.5411
Epoch 14/50 | Gen Loss: 1.0012 | Disc Loss: 0.9557
Epoch 15/50 | Gen Loss: 0.9091 | Disc Loss: 0.9906
Epoch 16/50 | Gen Loss: 0.9356 | Disc Loss: 1.0075
Epoch 17/50 | Gen Loss: 0.8839 | Disc Loss: 0.9884
Epoch 18/50 | Gen Loss: 1.1381 | Disc Loss: 0.7709
Epoch 19/50 | Gen Loss: 1.3258 | Disc Loss: 0.6409
Epoch 20/50 | Gen Loss: 1.3512 | Disc Lo

In [10]:
import gradio as gr


In [11]:

# Generate Image from Noise

generator = tf.keras.models.load_model("/content/mnist_gan_generator.keras")

def generate_image(seed):
    noise = tf.random.normal([1, 100], seed=seed)
    generated_image = generator(noise, training=False).numpy().reshape(28, 28)
    generated_image = (generated_image + 1) / 2.0  # Rescale to [0, 1]
    return generated_image


In [12]:
interface = gr.Interface(
    fn=generate_image,
    inputs=gr.Slider(minimum=0, maximum=10000, step=5, label="Random Seed"),
    outputs=gr.Image(height=280, width=280, image_mode='L', label="Generated Digit"),
    title="GAN Digit Generator",
    description="Generate handwritten digits using a trained GAN. Adjust the seed to change the output."
)

In [13]:
interface.launch(debug=False)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://695d55312723bf8811.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
